In [1]:
import os
from pinecone import Pinecone,ServerlessSpec
from dotenv import load_dotenv
import time


load_dotenv()


# Pinecone Configuration
api_key=os.getenv("PINECONE_API_KEY")
pc=Pinecone(api_key=api_key)

spec=ServerlessSpec(
    cloud="aws",region="us-east-1"
)

index_name="shop-product-catalog"

# connect to the index 
myindex=pc.Index(index_name)
time.sleep(1)

In [2]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings

load_dotenv()


# Google GenAI API
os.environ["GOOGLE_API_KEY"]=os.getenv("GOOGLE_API_KEY")
embed_model=GoogleGenerativeAIEmbeddings(model="models/embedding-001")

In [3]:
from langchain_pinecone import PineconeVectorStore

vectorstore=PineconeVectorStore(
    index=myindex,
    embedding=embed_model,
    text_key='Description'
)

In [4]:
query="What is the price of Superstar Bold 2 product?"

vectorstore.similarity_search(
    query,
    k=1
)

[Document(id='1042', metadata={'Gender': 'Men', 'Price': '11999', 'PrimaryColor': 'Black', 'ProductBrand': 'Adidas', 'ProductName': 'Superstar Bold 2'}, page_content='Classic basketball shoes with a modern twist and bold design')]

In [23]:
import os
import google.generativeai as genai

load_dotenv()

chat_history=[]


system_message=(
     "If a query lacks a direct answer e.g. durability, generate a response based on related features. "
    "You are a helpful and respectful shop assistant who answers queries relevant only to the shop. "
    "Please answer all questions politely. Use a conversational tone, like you're chatting with someone, "
    "not like you're writing an email. If the user asks about anything outside of the shop data like if they ask "
    "something irrelevant, simply say, 'I can only provide answers related to the shop, sir."
)

In [24]:
def gen_answer(system_message,chat_history,prompt):
    genai.configure(api_key=os.getenv("GOOGLE_API_KEY"))
    model=genai.GenerativeModel('gemini-1.5-flash')

    # append prompt to chat history
    chat_history.append(f"User: {prompt}")

    # combine system message to chat history
    full_prompt=f"{system_message}\n\n" + "\n".join(chat_history)+"\nAssistant:"

    # gen response
    response=model.generate_content(full_prompt).text
    chat_history.append(f"Assistant: {response}")

    return response

In [25]:
def get_relevant_chunk(query,vectorstore):
    results=vectorstore.similarity_search(query,k=1)
    if results:
        metadata=results[0].metadata
        context=(
            f"Product Name: {metadata.get('ProductName','Not Available')}\n"
            f"Brand: {metadata.get('Brand','Not Available')}\n"
            f"Price: {metadata.get('Price','Not Available')}\n"
            f"Color: {metadata.get('Color','Not Available')}\n"
            f"Description: {results[0].page_content}"
        )
        return context
    return "No relevant search"


def make_prompt(query,context):
    return f"Query: {query}\n\nContext:\n{context}\n\nAnswer:"

In [26]:
def main():
    query="What is the price of Superstar Bold 2 product? "
    relevant_text=get_relevant_chunk(query,vectorstore)
    prompt=make_prompt(query,relevant_text)

    answer=gen_answer(system_message,chat_history,prompt)
    print("Answer:",answer)

    query2="Can you tell me about the  design of this product?"
    relevant_text2=get_relevant_chunk(query2,vectorstore)
    prompt2=make_prompt(query2,relevant_text2)
    answer2=gen_answer(system_message,chat_history,prompt2)
    print(answer2)


if __name__=="__main__":
    main()

Answer: Assistant: Hi there!  The Superstar Bold 2s are priced at 11999.  Is there anything else I can help you with today?

Hi there! The Stan Smiths have a minimalist design, which is really classic and timeless.  Is there anything else I can help you with regarding the Stan Smiths, or perhaps other products?

